In [0]:
!pip install pyonmttok fasttext

     |████████████████████████████████| 2.2MB 4.9MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2384113 sha256=aad022b11381b311afcd4548f6f0750a6acc75b728358803992436d9ef5ec6e3
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext


In [0]:
!rm -f en_tg_train.tar.gz
!wget https://www.dropbox.com/s/umd8tyx4wz1wquq/en_tg_train.tar.gz
!rm -f en_tg_train.json
!tar -xzvf en_tg_train.tar.gz
!rm en_tg_train.tar.gz

--2020-01-04 18:25:52--  https://www.dropbox.com/s/erbi4w5fuacyvw9/en_tg_train.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/erbi4w5fuacyvw9/en_tg_train.tar.gz [following]
--2020-01-04 18:25:53--  https://www.dropbox.com/s/raw/erbi4w5fuacyvw9/en_tg_train.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc39dfb4487d544a5c9edec24ae7.dl.dropboxusercontent.com/cd/0/inline/AvhVptwLcoXRwgS-yKn0llcKFoUy1OXrS2QnqozAiqSADO5tCy5rdSxfWHxaU_m71Hhq8h1FQdWJylJB6rzqY5NBjQfabEKeX4c3sDd_Zu2e7WnXalY_UkS-eejQcJSOxV4/file# [following]
--2020-01-04 18:25:53--  https://uc39dfb4487d544a5c9edec24ae7.dl.dropboxusercontent.com/cd/0/inline/AvhVptwLcoXRwgS-yKn0llcKFoUy1OXrS2QnqozAiqSADO5tCy5rdSxfWHxaU_m71Hhq8h1FQdWJylJB6rzqY5NBjQfabEKe

In [0]:
# https://www.kaggle.com/pariza/bbc-news-summary/data

!rm -f bbc-news-summary.zip
!wget https://www.dropbox.com/s/gq76b24q3x5n1ku/bbc-news-summary.zip
!unzip bbc-news-summary.zip

--2020-01-04 18:26:08--  https://www.dropbox.com/s/gq76b24q3x5n1ku/bbc-news-summary.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/gq76b24q3x5n1ku/bbc-news-summary.zip [following]
--2020-01-04 18:26:09--  https://www.dropbox.com/s/raw/gq76b24q3x5n1ku/bbc-news-summary.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6710241e4039e964456982334f.dl.dropboxusercontent.com/cd/0/inline/AvhoEXmHUvIjJjHavCypNfD0XGSSvtdZi2z46d-ISRx6NYl62DVrUL7sqHykV1T8C7u_JGLdaHQLLDx2n_otqchO1_GiNNzwHr32jOHquzkngNFbN7H-7X44ahu3Lqfmqbw/file# [following]
--2020-01-04 18:26:09--  https://uc6710241e4039e964456982334f.dl.dropboxusercontent.com/cd/0/inline/AvhoEXmHUvIjJjHavCypNfD0XGSSvtdZi2z46d-ISRx6NYl62DVrUL7sqHykV1T8C7u_JGLdaHQLLDx2n_otqchO1_

In [0]:
# https://www.kaggle.com/rmisra/news-category-dataset

!rm -f news-category-dataset.zip
!wget https://www.dropbox.com/s/ua18htwqrkwnfpg/news-category-dataset.zip
!unzip news-category-dataset.zip

--2020-01-04 18:26:18--  https://www.dropbox.com/s/ua18htwqrkwnfpg/news-category-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ua18htwqrkwnfpg/news-category-dataset.zip [following]
--2020-01-04 18:26:18--  https://www.dropbox.com/s/raw/ua18htwqrkwnfpg/news-category-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3de4aca0e504c5a90dc764db3b.dl.dropboxusercontent.com/cd/0/inline/AvgyTcIJbWkkyemRqbY23ZhQzkqVfm5l0lok1vdkIkKv1Z2q8NZn0h8aw3PGOwviRrszpSBj4mGut2k5VfMz171MR8OORkyTJzEjhrCgeekysTR9tMq63BFMp9ln5_UQvN0/file# [following]
--2020-01-04 18:26:19--  https://uc3de4aca0e504c5a90dc764db3b.dl.dropboxusercontent.com/cd/0/inline/AvgyTcIJbWkkyemRqbY23ZhQzkqVfm5l0lok1vdkIkKv1Z2q8NZn0h8aw3PGOwviRrszpSBj4mG

In [0]:
# https://www.kaggle.com/snapcrack/all-the-news

!rm -f all-the-news.zip
!wget https://www.dropbox.com/s/bacg3cxckeqw6a9/all-the-news.zip
!unzip all-the-news.zip

--2020-01-04 18:26:25--  https://www.dropbox.com/s/bacg3cxckeqw6a9/all-the-news.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/bacg3cxckeqw6a9/all-the-news.zip [following]
--2020-01-04 18:26:26--  https://www.dropbox.com/s/raw/bacg3cxckeqw6a9/all-the-news.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd0d93972b46a8ae3f31d99388e.dl.dropboxusercontent.com/cd/0/inline/Avh64unANy1B0oQzA5jcQv4Alse51HaG9h4x16W6uk6Scz56H6c4z2vBjvVfo8DdcNKP2yweevnU0KLPjZ1H6N_5tsasK2YjmgQUnZmwoEljtR9waW7STdXT1IbxMkyjaBE/file# [following]
--2020-01-04 18:26:26--  https://ucd0d93972b46a8ae3f31d99388e.dl.dropboxusercontent.com/cd/0/inline/Avh64unANy1B0oQzA5jcQv4Alse51HaG9h4x16W6uk6Scz56H6c4z2vBjvVfo8DdcNKP2yweevnU0KLPjZ1H6N_5tsasK2YjmgQUnZ

In [0]:
import json

with open('en_tg_train.json', "r") as r:
    tg_train_data = json.load(r)

tg_titles = [record["title"] for record in tg_train_data]
tg_texts = [record["text"] for record in tg_train_data]
print(tg_titles[0])
print(tg_texts[0])
print(len(tg_titles))

Punjab as a driver for Pakistan’s growth
Several years ago, I was invited by the then government of Punjab to inaugurate a series of lectures funded by the World Bank on Punjab’s economic prospects. The Bank at that time was working on a study that studied Pakistan’s economic future. The institution was concerned that the country’s policymakers had not fully grasped Pakistan’s potential and were not working on realising it. In my talk and those that were to follow me, we were to give our take on how Punjab, the country’s most populous province and also its most advanced, could lead the rest of the country towards a higher growth trajectory. My lecture, held at the Pearl Continental Hotel, was well attended. In the audience were government officials, some business people and a few from the world of politics. The questions that followed my presentation showed some surprise at my optimism since I had suggested that with the adoption of the right set of policies, the provincial rate of gro

In [0]:
import os

def get_bbc_texts(input_directory):
    assert os.path.exists(input_directory)
    records = []
    for rubric_dir in os.listdir(input_directory):
        rubric_dir = os.path.join(input_directory, rubric_dir)
        if not os.path.isdir(rubric_dir):
            continue
        for file_name in os.listdir(rubric_dir):
            file_name = os.path.join(rubric_dir, file_name)
            with open(file_name, "r") as r:
                try:
                    content = r.read().replace("\n", " ")
                except Exception as e:
                    continue
                records.append(content)
    return records

bbc_texts = get_bbc_texts("BBC News Summary/News Articles")
print(bbc_texts[0])
print(len(bbc_texts))

EU China arms ban 'to be lifted'  The EU embargo on arms exports to China is likely to be lifted in the next six months despite US objections, UK Foreign Secretary Jack Straw has said.  The 15-year-old ban was imposed in the aftermath of China's crackdown on protesters in Tiananmen Square. Mr Straw told a Commons select committee human rights concerns over China remained. But he said it was wrong to put China under the same embargo as countries such as Zimbabwe and Burma.  In December, the EU pledged to work towards lifting the ban but said it was not ready to do so yet. The EU's move was welcomed at the time by Beijing, which described the embargo as a "product of the Cold War". German Chancellor Gerhard Schroeder and French President Jacques Chirac have repeatedly called for the embargo to be lifted. Britain has been more cautious on the issue, but on Wednesday Mr Straw said he also wanted it to end. "I have long understood China's argument, that to lump them in with, say, Burma and 

In [0]:
import json

nc_texts = []
with open("News_Category_Dataset_v2.json", "r") as r:
    for line in r:
        data = json.loads(line)
        title = data["headline"]
        text = data["short_description"]
        nc_texts.append(title + " " + text)
print(nc_texts[0])
print(len(nc_texts))

There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV She left her husband. He killed their children. Just another day in America.
200853


In [0]:
import csv
import sys
csv.field_size_limit(sys.maxsize)

all_the_news_files = ("articles1.csv", "articles2.csv", "articles3.csv")
atn_titles = []
atn_texts = []
for file_name in all_the_news_files:
    with open(file_name, "r") as r:
        next(r)
        reader = csv.reader(r, delimiter=',')
        for row in reader:
            _, _, title, _, _, _, _, _, _, text = row
            atn_titles.append(title)
            atn_texts.append(text)
print(atn_titles[0])
print(atn_texts[0])
print(len(atn_titles))

House Republicans Fret About Winning Their Health Care Suit - The New York Times
WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporaril

In [0]:
import pyonmttok
import random
tokenizer = pyonmttok.Tokenizer("conservative")

def preprocess(text):
    text = str(text).strip().replace("\n", " ").replace("\xa0", " ").lower()
    tokens, _ = tokenizer.tokenize(text)
    text = " ".join(tokens)
    return text

all_samples = tg_titles + tg_texts + bbc_texts + nc_texts + atn_titles + atn_texts
random.shuffle(all_samples)
processed_all_samples = [preprocess(text) for text in all_samples]
processed_all_samples = [text for text in processed_all_samples if text.strip()]
print(processed_all_samples[0])
print(len(processed_all_samples))

i sell fruits because it ’ s fun : 97-year-old granny in viral post sees wave of kindness
688405


In [0]:
# Clear RAM
del tg_titles
del tg_texts
del bbc_texts
del nc_texts
del all_samples
del atn_titles
del atn_texts

In [0]:
# Clear Disk
!rm -rf "BBC News Summary"
!rm -rf "bbc news summary"
!rm News_Category_Dataset_v2.json
!rm en_tg_train.json
!rm articles1.csv
!rm articles2.csv
!rm articles3.csv

In [0]:
with open("train.txt", "w", encoding="utf-8") as w:
    for sample in processed_all_samples:
        w.write(sample.strip() + "\n")

In [0]:
!tar -czvf en_unsupervised_train.tar.gz train.txt

train.txt


In [0]:
from fasttext import train_unsupervised

model = train_unsupervised('train.txt', model='skipgram', dim=50, epoch=10, minCount=50, bucket=200000, verbose=2)